# Matching Network handling

*NOTE* Suppose I have a MN of 5 component like this:

*-----L2-------------------C3---------------C1------------------*

           |             |             |
           
           L1             C2             Z_load
           
           |             |             |
      
*------------------------------------------------------------*

In [1]:
import numpy as np

Let's try to simplify the parallel or series type of a circuit

In [2]:
def parallel(Z1, Z2):
    return 1/(1/Z1 + 1/Z2)

def series(Z1, Z2): 
    return Z1 + Z2

Then, I assign type (R/L/C), connection orientation (ser/par) and values of 5 components, respectively:

In [3]:
num_component = 5

com1 = ('C','ser',10*1e-12)
com2 = ('C', 'par', 20*1e-12)
com3 = ('C', 'ser', 30*1e-12)
com4 = ('L', 'par', 40*1e-9)
com5 = ('L', 'ser', 50*1e-9)
components = (com1, com2, com3, com4, com5)
components[3][1]

'par'

Suppose I'm working @2GHz and Z_load is perfectly matched @50Ohm

In [4]:
freq = 1e9
Z_load = 50+1j*20

Lets make a function of output impedance from scratch. At first, try to change the circuit into impedances type

*-----ZL2-------------------ZC3---------------ZC1------------------*

           |             |             |
           
           ZL1             ZC2             Z_load
           
           |             |             |
      
*------------------------------------------------------------*

In [5]:
def out_impedance(num_component, components, freq, Z_load):
    Z = np.zeros((num_component+1,),dtype = complex)
    Z_out = np.zeros((num_component,),dtype = complex)
    # calculate impedance of each component
    Z[0] = Z_load
    for i in range(num_component):
        if components[i][0] == 'C':
            Z[i+1] = 1/(1j*2*np.pi*freq*components[i][2])
        if components[i][0] == 'L':
            Z[i+1] = 1j*2*np.pi*freq*components[i][2]
        else:
            Z[i+1] = components[i][2]
    # calculate result
    for i in range(num_component):
        if components[i][1] == 'ser':
            Z_out[i] = series(Z[i], Z[i+1])
        else:
            Z_out[i] = parallel(Z[i], Z[i+1])
    return Z_out[num_component-1]

Finally, simply call the function

In [6]:
out_impedance(num_component, components, freq, Z_load)

565.4866776461628j